In [1]:
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Activation, Embedding, Conv1D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt


from dataprocessing.find_top_movie_with_sequence import find_top_dataset
from dataprocessing.label_and_feature import getSortedClassContents, checkConsists, buildFeature, saveClassesToFile
from dataprocessing.time import checkingTimeDifferent
from recom_model import trainWithBidirectional,trainWithEmbeddingDense

[[[[0, 0, 2]], 4], [[[0, 0, 1]], 1], [[[0, 0, 3]], 5], [[[0, 0, 4]], 7], [[[0, 3, 1]], 3]]


In [2]:
MIN_CONTENTS_ON_USER = 250
MAX_DAYS = 7
MAX_SEQUENCE = 1
DATA_DIR = "data"

recommandation_df = pd.read_csv('{}/normalizedata.csv'.format(DATA_DIR)).sort_values(by=['time'])

In [3]:
sortedClassContents = getSortedClassContents(recommandation_df, MIN_CONTENTS_ON_USER)
saveClassesToFile(sortedClassContents)
lenSortedClassContents = len(sortedClassContents)
print(lenSortedClassContents)

[0, 1, 15, 23, 39, 69, 79, 162, 169, 202, 205, 213, 252, 255, 306, 308, 312, 322, 369, 371, 379, 434, 448, 455, 478, 479, 480, 481, 492, 507, 513, 535, 548, 623, 626, 640, 644, 657, 682, 683, 697, 700, 752, 772, 776, 777, 781, 824, 832, 839, 883, 913, 936, 960, 964, 977, 989, 993, 1006, 1020, 1026, 1115, 1137, 1185, 1187, 1193, 1239, 1242, 1256, 1269, 1304, 1310, 1397, 1415, 1420, 1428, 1429, 1433, 1436, 1437, 1490, 1498, 1506, 1512, 1521, 1534, 1554, 1559, 1567, 1578, 1579, 1589, 1591, 1599, 1602, 1606, 1607, 1609, 1619, 1620, 1624, 1641, 1652, 1659, 1677, 1700, 1716, 1719, 1756, 1758, 1774, 1777, 1781, 1788, 1790, 1791, 1795, 1798, 1806, 1807, 1813, 1819, 1823, 1826, 1830, 1833, 1834, 1836, 1838, 1839, 1840, 1845, 1847, 1853, 1855, 1862, 1863, 1872, 1873, 1876, 1877, 1879, 1885, 1896, 1910, 1912, 1913, 1927, 1928, 1935, 1936]
151


In [4]:
visitors_df = recommandation_df['visitor'].drop_duplicates()
maxItem = recommandation_df['Content'].max()
training_data = []
for index, item in visitors_df.iteritems():
    video = recommandation_df[recommandation_df['visitor'] == item]
    if video.size > 1:
        tempContents = []
        indexContent = 0
        for index, item in video['Content'].iteritems():
            if item not in sortedClassContents:
                continue
            if len(tempContents) > MAX_SEQUENCE:
                tempContents = tempContents[1:]
                continue
            if checkConsists(item, tempContents):
                continue
            if len(tempContents) > 0:
                nowDate = video['time'].iloc[indexContent]
                beforeDate = video['time'].iloc[indexContent - 1]

                if checkingTimeDifferent(nowDate, beforeDate, MAX_DAYS):
                    tempContents = []
                else:
                    feature = buildFeature(tempContents, sortedClassContents, MAX_SEQUENCE)
                    label = sortedClassContents.index(item)
                    training_data.append([[feature], label])
            tempContents.append(item)
            indexContent += 1

validation_data = find_top_dataset(training_data)
print("training size: ", len(training_data))
print("validation size: ", len(validation_data))
print(training_data)
print("-------------------------")
print(validation_data)
random.shuffle(training_data)

training size:  16678
validation size:  150
[[[[74]], 43], [[[54]], 82], [[[9]], 11], [[[18]], 47], [[[93]], 82], [[[68]], 69], [[[69]], 121], [[[121]], 26], [[[141]], 82], [[[82]], 111], [[[133]], 134], [[[134]], 86], [[[116]], 141], [[[133]], 106], [[[79]], 66], [[[46]], 82], [[[116]], 82], [[[46]], 77], [[[28]], 134], [[[38]], 39], [[[39]], 11], [[[11]], 65], [[[92]], 4], [[[4]], 89], [[[6]], 128], [[[46]], 77], [[[9]], 142], [[[108]], 62], [[[62]], 124], [[[124]], 57], [[[101]], 141], [[[132]], 127], [[[141]], 91], [[[82]], 81], [[[89]], 73], [[[141]], 23], [[[140]], 139], [[[73]], 134], [[[65]], 100], [[[117]], 147], [[[95]], 136], [[[15]], 70], [[[87]], 127], [[[127]], 10], [[[42]], 28], [[[28]], 9], [[[141]], 140], [[[140]], 82], [[[141]], 144], [[[47]], 134], [[[134]], 86], [[[86]], 27], [[[27]], 123], [[[123]], 95], [[[140]], 33], [[[33]], 111], [[[111]], 121], [[[121]], 144], [[[144]], 133], [[[119]], 141], [[[133]], 144], [[[111]], 15], [[[133]], 131], [[[131]], 41], [[[41]]

In [5]:
features = []
labels = []
for feature, label in training_data:
    features.append(feature)
    labels.append(label)

In [32]:
X = np.array(features)
Y = np.array(labels)
print(X.shape)



(16678, 1, 1)


In [33]:
features_val = []
labels_val = []
for feature, label in validation_data:
    features_val.append(feature)
    labels_val.append(label)

X_val = np.array(features_val)
Y_val = np.array(labels_val).astype(np.float32)

In [34]:
# for cnn
X = np.squeeze(X, axis=1)
print(X.shape)
model = Sequential()   
model.add(Dense(1024, activation="relu", input_shape=(X.shape[1:])))
model.add(Dense(1024, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(lenSortedClassContents))
model.add(Activation("softmax"))
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])


(16678, 1)


In [ ]:
# for embedding Dense
X = np.squeeze(X, axis=1)
model = trainWithEmbeddingDense(MAX_SEQUENCE, lenSortedClassContents)

In [ ]:
name = "testing-1"
tensorboard = TensorBoard(log_dir='logs/{}'.format(name))
history = model.fit(X, Y, epochs=250, batch_size=64, verbose=1,validation_data=(X_val, Y_val), callbacks=[tensorboard])

Epoch 1/250
261/261 [==============================] - 1s 5ms/step - loss: 4.8399 - accuracy: 0.0279 - val_loss: 4.6882 - val_accuracy: 0.0267
Epoch 2/250
261/261 [==============================] - 1s 5ms/step - loss: 4.7862 - accuracy: 0.0295 - val_loss: 4.6113 - val_accuracy: 0.0400
Epoch 3/250
261/261 [==============================] - 1s 5ms/step - loss: 4.7430 - accuracy: 0.0342 - val_loss: 4.5224 - val_accuracy: 0.0400
Epoch 4/250
261/261 [==============================] - 1s 5ms/step - loss: 4.6962 - accuracy: 0.0419 - val_loss: 4.3615 - val_accuracy: 0.0600
Epoch 5/250
261/261 [==============================] - 1s 5ms/step - loss: 4.6563 - accuracy: 0.0434 - val_loss: 4.2797 - val_accuracy: 0.1067
Epoch 6/250
261/261 [==============================] - 1s 5ms/step - loss: 4.6354 - accuracy: 0.0484 - val_loss: 4.2667 - val_accuracy: 0.0800
Epoch 7/250
261/261 [==============================] - 1s 5ms/step - loss: 4.6171 - accuracy: 0.0515 - val_loss: 4.1527 - val_accuracy: 0.0733

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')


In [ ]:
plot_graphs(history, 'loss')

In [ ]:
plot_graphs(history, 'val_loss')

In [ ]:
plot_graphs(history, 'val_accuracy')

In [ ]:
def get5TopPredict(predict):
    predicts = predict[0].argsort()[-5:][::-1]
    for i in predicts:
        print(sortedClassContents[i])
        

In [ ]:
      
def predictDense(first):
    predict = model.predict([[sortedClassContents.index(first)]])
    get5TopPredict(predict)
    return sortedClassContents[np.argmax(predict[0])]


In [ ]:
predictResult = predictDense(1756)
print(predictResult)

In [ ]:
run_model = tf.function(lambda x: model(x))
# This is important, let's fix the input size.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1,6], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "model"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)


In [ ]:
MODEL_DIR = "model"
model.save(MODEL_DIR) 

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()

In [ ]:
# Run the model with TensorFlow to get expected results.
TEST_CASES = 10

# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)
print(input_details[0]["index"])
for i in range(TEST_CASES):
  expected = model.predict([[i]])
  interpreter.set_tensor( input_details[0]["index"], np.array([[i]]).astype(np.float32))
  interpreter.invoke()
  result = interpreter.get_tensor(output_details[0]["index"])

  # Assert if the result of TFLite model is consistent with the TF model.
  np.testing.assert_almost_equal(expected, result)
  print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

  # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
  # the states.
  # Clean up internal states.
  interpreter.reset_all_variables()


In [ ]:
open("tf_model/converted_model.tflite", "wb").write(tflite_model)

In [ ]:
interpreter.set_tensor( input_details[0]["index"], np.array([[sortedClassContents.index(700)]]).astype(np.float32))
interpreter.invoke()
result = interpreter.get_tensor(output_details[0]["index"])
print(sortedClassContents[np.argmax(result)])